## Fine-tuningしたBiTで推論を試す

In [1]:
import pathlib
import json
import logging

import tensorflow as tf
import tensorflow_hub as tfhub


tf.get_logger().setLevel(logging.WARNING)
for d in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(d, True)

In [2]:
tf.__version__, tfhub.__version__

('2.3.0', '0.8.0')

推論対象の画像をデータセット化  

ファインチューニング時の Crop size にリサイズする。

In [3]:
target_dir = pathlib.Path('test').resolve()

def generator():
    path_iter = filter(lambda p: p.stem.isdigit() and p.suffix in {'.png', '.jpg'}, target_dir.iterdir())
    for path in sorted(path_iter, key=lambda p: int(p.stem)):
        image = tf.image.decode_image(tf.io.read_file(str(path)))
        image = tf.image.resize(image, [480, 480])
        image = image / 255.
        yield image, tf.constant(path.name)

batch = 32
ds = tf.data.Dataset.from_generator(generator, (tf.float32, tf.string))
ds = ds.batch(batch).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

model_dir = pathlib.Path('models/m-r50x1/2').resolve()
labels = json.loads((model_dir / 'label_names.json').read_text())
model = tfhub.KerasLayer(str(model_dir / 'model'), trainable=False)

In [4]:
from IPython.display import JSON

for images, ids in ds.take(1):
    logits = model(images)

    ids = ids.numpy()
    indices = tf.math.argmax(logits, axis=1).numpy()

    display(JSON({i.decode(): labels[j] for i, j in zip(ids, indices)}))

<IPython.core.display.JSON object>